# Домашнее задание 3

Многие настольные игры используют гексагональные сетки для игры

![Фото гексагональной сетки](https://i2022.otzovik.com/2022/02/07/13040498/img/2052370_34559663.jpeg)

В этом домашнем задании рассмотрим как хранятся гексагоны, как можно генерировать ландшафт и порешаем задачи поиска кратчайшего пути м-ду точками игрового поля

## Как хранить гексагоны (1 балл)

[Тут описана матчасть](https://www.redblobgames.com/grids/hexagons/)

Самые удобные варианты - матрица и куб

Как отрисовать гексагоны посредством матрицы - нетривиальная задача, поэтому ниже рассматривается хранение через куб

У каждой точки в кубе есть 6 смежных точек, как сторон в гексагоне

Представим что каждый гексагон - это некоторая точка в пространстве (в кубе)

<img src="https://drive.google.com/uc?export=view&id=1Nnii4DSWV_tk-r1X-LGHcw0PM_4t2z3X" height="200"/>

Попробуем к нашему гексагону приписать смежные точки

<img src="https://drive.google.com/uc?export=view&id=164DoK1j1ojL9TuAKu4etxbetXGR4_Vj5" height="400"/>

Но заметим что смежные точки не смежны м-ду собой

Например, (0, -1, 0) и (0, 0, -1) - не соседи, хотя в этой модели они стоят рядом

Поэтому такая модель не будет работать

Попробуем другую модель

Введем такие правила:
- Сумма координат равна нулю
- x не изменяется по вертикали
- y и z не изменяются по соответствующим диагоналям

<img src="https://drive.google.com/uc?export=view&id=1sPOFOJXsX9Ppd2QpYXmxv-2Z6kRHw4VR" height="400"/>

Теперь немного поменялось понятие смежности

Смежными считаются клетки, отличающиеся по **двум** координатам и отличающиеся ровно на **два** пункта

Например:
(-1, 1, 0) и (-1, 0, 1)

Отличаются координатами y и z, сумма разниц координат равна двум

Теперь модель работает

Распишем по аналогии для гексагонов длины 2 от центра

<img src="https://drive.google.com/uc?export=view&id=10bhJJ-AD3ltGN6KUYQEo_tKjnqdSrD9D" height="600"/>

Однако использовать отрицательные индексы не стоит, будем использовать только положительные

Для этого нам придется сместить все координаты

Нужно перевести координаты из диапазона [-2; 2] в диапазон [0; 4], значит к каждой координате надо прибавить 2

Для хранения поля с расстоянием 2 от центра, нам нужен куб размера 5 x 5 x 5 (диапазон [0; 4])

Соответственно, куб размера N x N x N, может хранить гексагоны с расстоянием до N // 2 от центра


Для того, чтобы хранить гексагон сам по себе в такой модели, нам нужен куб

Но помимо этого гексагон еще должен хранить какую-то полезную информацию, значит нам нужна как минимум 4-мерная фигура

Работать с таким очень тяжело, поэтому будем хранить некоторый гексагон как массив длины 4 в формате\
[x, y, z, data]

А куб со всеми гексагонами будем хранить как матрицу, каждый элемент которой - гексагон (длины 4)

Для начала сделаем куб 5 x 5 x 5

In [1]:
import numpy as np
from itertools import product

N = 5

"""
TODO:
- Сгенерировать матрицу с shape (N**3, 4)
- Сгенерить декартово произведение координат длины 3
- Заполнить 4-ю колонку дефолтными значениями (-1, например)

Результат - data
"""
data = np.zeros((N**3, 4), dtype=np.int32)
data[:, :-1] = np.array(list(product(range(N), range(N), range(N))))
data[:, -1] = -1

In [2]:
data

array([[ 0,  0,  0, -1],
       [ 0,  0,  1, -1],
       [ 0,  0,  2, -1],
       [ 0,  0,  3, -1],
       [ 0,  0,  4, -1],
       [ 0,  1,  0, -1],
       [ 0,  1,  1, -1],
       [ 0,  1,  2, -1],
       [ 0,  1,  3, -1],
       [ 0,  1,  4, -1],
       [ 0,  2,  0, -1],
       [ 0,  2,  1, -1],
       [ 0,  2,  2, -1],
       [ 0,  2,  3, -1],
       [ 0,  2,  4, -1],
       [ 0,  3,  0, -1],
       [ 0,  3,  1, -1],
       [ 0,  3,  2, -1],
       [ 0,  3,  3, -1],
       [ 0,  3,  4, -1],
       [ 0,  4,  0, -1],
       [ 0,  4,  1, -1],
       [ 0,  4,  2, -1],
       [ 0,  4,  3, -1],
       [ 0,  4,  4, -1],
       [ 1,  0,  0, -1],
       [ 1,  0,  1, -1],
       [ 1,  0,  2, -1],
       [ 1,  0,  3, -1],
       [ 1,  0,  4, -1],
       [ 1,  1,  0, -1],
       [ 1,  1,  1, -1],
       [ 1,  1,  2, -1],
       [ 1,  1,  3, -1],
       [ 1,  1,  4, -1],
       [ 1,  2,  0, -1],
       [ 1,  2,  1, -1],
       [ 1,  2,  2, -1],
       [ 1,  2,  3, -1],
       [ 1,  2,  4, -1],


Сделаем функцию изменения массива data по координатам гексагона

Полезной информацией в гексагоне будем считать его расстояние от центра куба

In [3]:
"""
coords - координаты в формате [x, y, z]
info - растояние гексагона от центра

TODO:
- Посчитать индекс координат в массиве
- По посчитанном индексу записать info
"""
def set_data(coords: list, info: int):
  index = coords[0]*N**2 + coords[1]*N + coords[2]
  data[index, -1] = info

Поменяем расстояние для длины 0 и 1



In [4]:
set_data([2, 2, 2], 0)
set_data([2, 3, 1], 1)
set_data([2, 1, 3], 1)
set_data([1, 2, 3], 1)
set_data([3, 2, 1], 1)
set_data([1, 3, 2], 1)
set_data([3, 1, 2], 1)

Сделаем аналогичную функцию, но центром будем считать гексагон с координатами [0, 0, 0]

In [5]:
"""
Делаем аналогичную вещь, но смещаем каждую координату
"""

def set_rel_data(coords: list, info: int):
  shifted_coords = [np.abs(coord - center_coord) for coord, center_coord in zip(coords, [2, 2, 2])]
  index = shifted_coords[0]*N**2 + shifted_coords[1]*N + shifted_coords[2]
  data[index, -1] = info

Поменяем расстояние для гексагонов длины 2

In [6]:
set_rel_data([0, 2, -2], 2)
set_rel_data([1, 1, -2], 2)
set_rel_data([2, 0, -2], 2)
set_rel_data([2, -1, -1], 2)
set_rel_data([2, -2, 0], 2)
set_rel_data([1, -2, 1], 2)
set_rel_data([0, -2, 2], 2)
set_rel_data([-1, -1, 2], 2)
set_rel_data([-2, 0, 2], 2)
set_rel_data([-2, 1, 1], 2)
set_rel_data([-2, 2, 0], 2)
set_rel_data([-1, 2, -1], 2)

Отрисуем нашу фигуру

In [7]:
import plotly.express as px
fig = px.scatter_3d(
    data,
    x=data[:, 0],
    y=data[:, 1],
    z=data[:, 2],
    color=data[:, 3]
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1v2MrbIptn1VUcQje7XCCz2-kdf_OpOgU)
</details>

Как видно, в фигуре слишком много "лишних" точек

Отрисуем только гексагоны (где в 4-й колонке не -1)

In [8]:
data_to_write = data[data[:, 3] != -1]
fig = px.scatter_3d(
    data_to_write,
    x=data_to_write[:, 0],
    y=data_to_write[:, 1],
    z=data_to_write[:, 2],
    color=data_to_write[:, 3]
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>
  
  ![График](https://drive.google.com/uc?export=view&id=1BT6oMWwbX2pb5bz2vgV_S55saad6J_MZ)
</details>

Сделаем функцию создания куба, хранящего гексагоны до заданного расстояния

In [9]:
"""
max_dist - максимальное расстояние гексагона

TODO:
- Вычислить N в зависимости от max_dist
- Собрать код генерации куба выше в одну функцию

Примечание: расстояние от центра не добавлять, займемся этим ниже
"""

def create_map(max_dist: int):
  N = 2 * max_dist + 1
  data = np.zeros((N**3, 4), dtype=np.int32)
  data[:, :-1] = np.array(list(product(range(N), range(N), range(N))))
  data[:, -1] = 1
  return data

In [10]:
MAX_DIST = 20
N = 2 * MAX_DIST + 1
map_data = create_map(MAX_DIST)

In [11]:
map_data

array([[ 0,  0,  0,  1],
       [ 0,  0,  1,  1],
       [ 0,  0,  2,  1],
       ...,
       [40, 40, 38,  1],
       [40, 40, 39,  1],
       [40, 40, 40,  1]], dtype=int32)

Благодаря правилу\
"сумма координатов гексагона равна 0", гексагоны лежат в плоскости, которая задается уравнением

$$ x + y + z = 0 $$

И для того, чтобы получить индексы гексагонов, достаточно проверять, удовлетворяет ли точка в кубе этому условию

Однако мы смещали каждую координату на некоторую длину, чтобы не использовать отрицательные индексы\
Значит, нам необходимо проверять условие

$$ (x - c) + (y - c) + (z - c) = 0 $$
$$ x + y + z = 3c $$

Где $c$ - размер смещения (т.е максимальная длина гексагона)

Сделаем функцию получения индексов гексагонов

In [12]:
"""
coords_map - куб с гексагонами

Функция проходит по точкам куба и проверяет соответствие условию гексагона
Возвращает массив индексов гексагонов

Для реализации лучше использовать np.where
"""

def get_hex_idxs(coords_map):
  return np.argwhere(coords_map[:, 0] + coords_map[:, 1] + coords_map[:, 2] == 3*MAX_DIST).flatten()

In [13]:
hexagon_idxs = get_hex_idxs(map_data)

hexagon_idxs

array([  860,   900,   940, ..., 67980, 68020, 68060])

Для каждого гексагона определим и запишем его расстояние до центра

Рекомендую еще раз взглянуть на рисунок и самостоятельно найти закономерность, по которой можно определять расстояние до центра

<img src="https://drive.google.com/uc?export=view&id=10bhJJ-AD3ltGN6KUYQEo_tKjnqdSrD9D" height="600"/>

In [14]:
hexagon_data = map_data[hexagon_idxs]
CENTER = np.array([MAX_DIST, MAX_DIST, MAX_DIST])
for hexagon in hexagon_data:
  hexagon[-1] = np.sum(np.abs(hexagon[:-1] - CENTER))

Нарисуем гексагоны с учетом расстояния до центра

In [15]:
fig = px.scatter_3d(
    hexagon_data,
    x=hexagon_data[:, 0],
    y=hexagon_data[:, 1],
    z=hexagon_data[:, 2],
    color=hexagon_data[:, 3]
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1tGj0jFt2EqfLOsqP7xFYnFDkeLHWECa1)
</details>

## Генерация игрового поля (1 балл)

До этого мы хранили в гексагоне его расстояние до центра

Теперь в гексагоне будем хранить тип клетки на поле (луг/река/гора и т.д.)

Для начала сделаем пустое поле

In [16]:
new_map = create_map(MAX_DIST)
hex_idxs = get_hex_idxs(new_map)
hex_data = new_map[hex_idxs]

In [17]:
fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    color=hex_data[:, 3]
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=11RfLICjhr5aUbHWFZbMp4X1LeG9IxYWL)
</details>

Для начала выпишем все возможные "движения" по гексогональной сетке

In [18]:
ALL_MOVEMENTS = np.array([
    [0, 1, -1],
    [1, 0, -1],
    [1, -1, 0],
    [0, -1, 1],
    [-1, 0, 1],
    [-1, 1, 0]
])

### Генерация рек

Для генерации рек:
- выберем 3 движения в одном направлении
- зададим число рек для генерации
- укажем тип клетки реки (10)
- Для каждой реки:
  - генерируем случайную длину реки (до 10 клеток)
  - выбираем случайную стартовую точку реки
  - до тех пор пока длина реки меньше выбранной
    - выбираем случайное движение из 3
    - передвигаемся к новой клетке (если клетка - поле)
    - меняем тип новой клетки на реку

In [19]:
import random

river_movements = ALL_MOVEMENTS[0:2]

RIVERS_COUNT = 40

MAX_RIVER_LENGTH = 7

RIVER_TYPE = 10

np.random.seed(42)
random.seed(42)

river_map = np.copy(new_map)

In [20]:
for i in range(RIVERS_COUNT):
  river_len = random.randint(0, MAX_RIVER_LENGTH)
  point_num = random.choice(hex_idxs)

  river_map[point_num, -1] = RIVER_TYPE
  l = 0
  while l < river_len:
    point_coords = river_map[point_num]
    move_coords = random.choice(river_movements)
    new_point_coords = point_coords[:-1] + move_coords
    new_point_num = new_point_coords[0]*N**2 + new_point_coords[1] * N + new_point_coords[2]
    if new_point_num in hex_idxs:
      river_map[new_point_num, -1] = RIVER_TYPE
      l += 1
      point_num = new_point_num
    else:
      break

Отрисуем получившееся поле

In [21]:
hex_data = river_map[hex_idxs]

fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    color=hex_data[:, 3]
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1RDB5zt5YDP2RUj0G-Z6a9kweqKehBYed)
</details>

### Генерация гор

Для генерации гор:
- зададим число гор для генерации
- укажем тип клетки горы (100)
- Для каждой горы:
  - генерируем случайную длину реки (до 10 клеток)
  - выбираем случайную точку карты
  - пробегаемся по всем движениям, для каждой клетки указываем тип поля - гора

In [22]:
hills_map = np.copy(river_map)

HILLS_COUNT = 10

HILLS_TYPE = 100

In [23]:
for i in range(HILLS_COUNT):
  point_num = random.choice(hex_idxs)
  point_coords = hills_map[point_num]
  hills_map[point_num, -1] = HILLS_TYPE
  for move_coords in ALL_MOVEMENTS:
    new_point_coords = point_coords[:-1] + move_coords
    new_point_num = new_point_coords[0]*N**2 + new_point_coords[1] * N + new_point_coords[2]
    if new_point_num in hex_idxs:
      hills_map[new_point_num, -1] = HILLS_TYPE

Покрасим клетки нашего поля в зависимости от типа клетки

In [24]:
PLAIN_TYPE = 1

types_map = {
    PLAIN_TYPE: "Plain",
    HILLS_TYPE: "Hill",
    RIVER_TYPE: "River",
}

vfunc = np.vectorize(types_map.get)

In [25]:
hex_data = hills_map[hex_idxs]

colors = vfunc(hex_data[:, 3])

fig = px.scatter_3d(
    hex_data,
    x=hex_data[:, 0],
    y=hex_data[:, 1],
    z=hex_data[:, 2],
    hover_name=hex_idxs,
    color=colors,
    color_discrete_sequence=["#387C44", "#3EA99F", "gray"],
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1Ccngh0NwwXGxqm9yUMwLVW82jlvnddgl)
</details>

## Алгоритмы поиска пути

### Подготовка

Покрасим клетки поля в тип поля "путь", чтобы увидеть какой путь был найден

Cоберем функцию для визуализации карты с путем

In [26]:
ROUTE_TYPE = 1000

def visualize_solve(route_path):
  route_map = np.copy(hills_map)

  for cell_idx in route_path:
    route_map[cell_idx][-1] = ROUTE_TYPE

  hex_data = route_map[hex_idxs]
  vfunc = np.vectorize(types_map.get)

  colors = vfunc(hex_data[:, 3])

  fig = px.scatter_3d(
      hex_data,
      x=hex_data[:, 0],
      y=hex_data[:, 1],
      z=hex_data[:, 2],
      hover_name=hex_idxs,
      color=colors,
      color_discrete_sequence=["#387C44", "#3EA99F", "gray", "maroon"],
  )
  fig.show()

In [27]:
visualize_solve([59_460, 57_780, 57_740, 56_060, 54_380])

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1qPivDUWiO7h66_apjgMJFkNRTDUQ9H9M)
</details>

### Бэктрекинг (1 балл)

Начнем с переборного алгоритма

Возьмем 2 точки карты и напишем решение бэктрекингом

In [28]:
back_from_idx = 52_940
back_to_idx = 45_860

Чтобы решение бэктрекингом не переполняло стек вызовов, введем 2 условности:

- Если расстояние м-ду текущей точкой и точкой назначения больше изначального расстояния (расстояния м-ду стартовой и конечной точкой) более чем в 2 раза - режем ветку решения

- Если расстояние от текущей больше чем от предыдущей точки - режем ветку решения

Сделаем функцию подсчета декартового расстояния

In [29]:
import math

def count_dist(a_idx, b_idx):
  a_data = hills_map[a_idx][:3]
  b_data = hills_map[b_idx][:3]

  return math.sqrt(sum((a_data - b_data)**2))

In [30]:
count_dist(back_from_idx, back_to_idx)

16.30950643030009

Напишем бэктрекинговое решение с учетом наших двух условностей

In [31]:
init_dist = count_dist(back_from_idx, back_to_idx)

"""
curr_idx - текущий индекс клетки
path - массив, путь до текущей клетки
prev_dist - расстояние от предыдущей клетки до итоговой
prev_movement_idx - индекс прошлого движения, используем чтобы не ходить "туда-сюда"
"""
def backtracking(curr_idx, path, prev_dist, prev_movement_idx):

  if curr_idx == back_to_idx:
    return path

  curr_dist = count_dist(curr_idx, back_to_idx)

  if curr_dist > 2 * init_dist or curr_dist > prev_dist:
    return None

  curr_point_coords = hills_map[curr_idx][:-1]
  prev_movement = ALL_MOVEMENTS[prev_movement_idx]

  for move_idx, move_coords in enumerate(ALL_MOVEMENTS):
    if prev_movement_idx != -1 and all([c == -p for c, p  in zip(move_coords, prev_movement)]):
      continue

    next_point_coords = curr_point_coords + move_coords
    next_idx = next_point_coords[0]*N**2 + next_point_coords[1] * N + next_point_coords[2]
    if next_idx in path:
      continue

    result = backtracking(next_idx, path + [next_idx], curr_dist, move_idx)
    if result is not None:
      return result
  return None

In [32]:
back_path = backtracking(back_from_idx, [back_from_idx], init_dist, -1)
back_path

[52940,
 54580,
 56220,
 57860,
 57820,
 57780,
 57740,
 57700,
 57660,
 57620,
 57580,
 57540,
 57500,
 57460,
 55780,
 55820,
 54140,
 54180,
 52500,
 50820,
 50860,
 49180,
 47500,
 47540,
 45860]

In [33]:
visualize_solve(back_path)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1bI_7zfWiRvJOf8__hiPL1d2SORnhDCKB)
</details>

Однако такое решение все еще может выйти за стек вызовов и не учитывает ландшафт

Соберем функцию визуализации

In [34]:
def visualise_backtrack_solve(idx_from, idx_to):
  init_dist = count_dist(idx_from, idx_to)

  back_path = backtracking(idx_from, [idx_from], init_dist, -1)

  if back_path is not None:
    visualize_solve(back_path)

P.S ячейка должна упасть из-за выхода за стек вызовов

In [35]:
visualise_backtrack_solve(29_380, 46_180)

### Жадный алгоритм (1 балл)

Раз бэктрекингом решить не получилось, попробуем сформулировать жадное решение

На самом деле, мы уже начали это делать в бэктрекинге (ввели ограничение на уменьшение расстояния)

Попробуем для начала такое решение:

На каждом шаге будем выбирать такое решение, которое уменьшает расстояние м-ду текущей и конечной точками

In [36]:
greedy_path = [back_from_idx]
curr_idx = back_from_idx
init_dist = count_dist(back_from_idx, back_to_idx)

curr_dist = init_dist
curr_data = hills_map[curr_idx]
coords = np.array(curr_data[:3])

while curr_dist > 0:
  min_dist = 1e9
  next_point = None
  for move_idx, move_coords in enumerate(ALL_MOVEMENTS):
    next_point_coords = coords + move_coords
    next_idx = next_point_coords[0]*N**2 + next_point_coords[1] * N + next_point_coords[2]
    if next_idx in greedy_path or hills_map[next_idx, -1] != PLAIN_TYPE:
      continue
    next_dist = count_dist(next_idx, back_to_idx)
    if min_dist > next_dist:
      min_dist = next_dist
      next_point = next_idx
  curr_dist = min_dist
  coords = hills_map[next_point][:-1]
  greedy_path.append(next_point)

In [37]:
greedy_path

[52940,
 52900,
 52860,
 52820,
 52780,
 52740,
 52700,
 51020,
 50980,
 49300,
 49260,
 47580,
 47540,
 45860]

In [38]:
visualize_solve(greedy_path)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1bI_7zfWiRvJOf8__hiPL1d2SORnhDCKB)
</details>

Соберем решение в одну функцию

In [39]:
def count_greedy_path(idx_from, idx_to):
  path = [idx_from]
  curr_idx = idx_from
  init_dist = count_dist(idx_from, idx_to)

  curr_dist = init_dist
  curr_data = hills_map[curr_idx]
  coords = np.array(curr_data[:3])

  while curr_dist > 0 and curr_idx != idx_to:
    min_dist = 1e9
    next_point = None
    for move_idx, move_coords in enumerate(ALL_MOVEMENTS):
      next_point_coords = coords + move_coords
      next_idx = next_point_coords[0]*N**2 + next_point_coords[1] * N + next_point_coords[2]
      if next_idx in greedy_path:
        continue
      next_dist = count_dist(next_idx, idx_to)
      if min_dist > next_dist:
        min_dist = next_dist
        next_point = next_idx
    curr_dist = min_dist
    curr_idx = next_point
    coords = hills_map[curr_idx][:-1]
    path.append(curr_idx)
  return path

Сделаем функцию визуализации

In [40]:
def visualise_greedy_solve(idx_from, idx_to):
  greedy_path = count_greedy_path(idx_from, idx_to)
  print(greedy_path)

  visualize_solve(greedy_path)

In [41]:
visualise_greedy_solve(29_540, 46_220)

[29540, 31220, 32900, 34580, 36260, 37940, 39580, 41260, 42900, 44580, 46220]


<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1HdTa3JMVQgpKA21lzC-9AMFvokW21qbt)
</details>

Однако мы все еще не учитываем ландшафт

Модифицируем задачу:
- На каждом шаге получаем все возможные решения
- Выбрасываем решения, которые идут через реки и горы
- Выбрасываем решения, которые идут через уже посещенные точки (убираем циклы)
- Из оставшихся решений выбираем решение с наименьшим расстоянием


In [42]:
def count_greedy_path(idx_from, idx_to):
  path = [idx_from]
  curr_idx = idx_from
  init_dist = count_dist(idx_from, idx_to)

  curr_dist = init_dist
  curr_data = hills_map[curr_idx]
  coords = np.array(curr_data[:3])

  while curr_dist > 0 and curr_idx != idx_to:
    min_dist = 1e9
    next_point = None
    for move_idx, move_coords in enumerate(ALL_MOVEMENTS):
      next_point_coords = coords + move_coords
      next_idx = next_point_coords[0]*N**2 + next_point_coords[1] * N + next_point_coords[2]
      if next_idx in greedy_path or hills_map[next_idx][-1] != PLAIN_TYPE:
        continue
      next_dist = count_dist(next_idx, idx_to)
      if min_dist > next_dist:
        min_dist = next_dist
        next_point = next_idx
    curr_dist = min_dist
    curr_idx = next_point
    coords = hills_map[curr_idx][:-1]
    path.append(curr_idx)
  return path

In [43]:
def visualise_greedy_solve(idx_from, idx_to):
  greedy_path = count_greedy_path(idx_from, idx_to)
  # for idx in greedy_path:
  #   print(idx, hills_map[idx])

  visualize_solve(greedy_path)

In [44]:
visualise_greedy_solve(29_540, 46_220)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1uOr-9bYE0Q_CnuLWGoXnY0X7fIQ-l9iX)
</details>

In [45]:
visualise_greedy_solve(29_380, 58_060)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1xNQJLnco2r6N6l0pTl3KZb1CWdqJMnpC)
</details>

Это решает задачу куда лучше, чем брутфорс и бэктрекинг, но не гарантирует оптимальное решение

### Динамическое программирование (1 балл)

Возьмем некоторый участок карты и попробуем найти путь м-ду точками при помощи дин. программирования

In [46]:
part_idxs = hex_idxs[hex_idxs > 51_000]

part_hex_data = hills_map[part_idxs]

colors = vfunc(part_hex_data[:, 3])

fig = px.scatter_3d(
    hex_data,
    x=part_hex_data[:, 0],
    y=part_hex_data[:, 1],
    z=part_hex_data[:, 2],
    hover_name=part_idxs,
    color=colors,
    color_discrete_sequence=["#387C44", "#3EA99F", "gray"],
)
fig.show()

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1yvtF-10oaWns4zoeqTGn4jjrB_LLqozt)
</details>

Попробуем найти путь м-ду левой верхней точкой и точкой справа снизу, как решали задачку "черепашки"

Зададим индексы искомых точек

In [47]:
from_idx = 64_620
to_idx = 52_140

In [48]:
visualize_solve([from_idx, to_idx])

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1s1ySJ0cebLaKeO0ALr8LtV7HffTAlO31)
</details>

Зададим движения по карте для этих двух точек

In [49]:
possible_movements = np.array([
    [1, 0, -1],
    [0, 1, -1],
])

Теперь зададим размер матрицы (посчитаем какое максимальное кол-во движений нужно сделать чтобы дойти от одного края матрицы до другого)

In [50]:
part_n = 8
part_m = 19

Сделаем матрицу весов передвижений

In [51]:
weights = np.full((part_n, part_m), PLAIN_TYPE)

weights

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

Решим задачку "черепашки"

P.S. Чтобы получить ячейку (i, j) нужно сделать i первых движений и j вторых движений

In [52]:
for i in range(part_n):
    for j in range(part_m):
        coords = hills_map[from_idx][:-1] - i * possible_movements[0] - j * possible_movements[1]
        idx = coords[0]*N**2+coords[1]*N + coords[2]
        weights[i, j] = hills_map[idx][-1]

for i in range(part_n):
    for j in range(part_m):
        if i == 0 and j == 0:
          continue
        if i == 0:
          weights[i, j] +=  weights[i, j - 1]
        elif j == 0:
          weights[i, j] +=  weights[i - 1, j]
        else:
          weights[i, j] += min(weights[i - 1, j], weights[i, j - 1])

In [53]:
weights

array([[  1,   2,   3,  13,  14,  24,  34,  44,  54,  55,  56,  57,  58,
         59,  60,  61,  62,  63,  64],
       [  2,   3,   4,  14,  24,  34,  44,  54,  55,  56,  57,  58,  59,
         60,  61,  62,  63,  64,  65],
       [  3,   4,   5,   6,   7,   8,   9,  10,  20,  21,  22,  23,  24,
         25,  26,  36,  37,  38,  39],
       [  4,   5,   6,   7,   8,   9,  10,  11,  21,  22,  23,  24,  25,
        125, 126,  37,  38,  39,  40],
       [ 14,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16, 116,
        216, 226,  38,  48,  49,  50],
       [ 24,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17, 117,
        217, 218,  39,  40,  41,  42],
       [ 25,  17,  18,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,
         20,  21,  22,  23,  24,  25],
       [ 26,  18,  19,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,
         21,  22,  23,  24,  25,  26]])

Найдем путь от начальной ячейки к конечной

In [54]:
curr_i = 0
curr_j = 0

n, m = weights.shape
moves = []
while curr_i != n - 1 or curr_j != m - 1:
  min_cost = 1e9
  next_move = None
  for move in possible_movements:
    i, j = curr_i + move[0], curr_j + move[1]
    if i < n and j < m:
      cost = weights[i, j]
      if min_cost > cost:
        min_cost = cost
        next_move = move
  curr_i, curr_j = curr_i + next_move[0], curr_j + next_move[1]
  # moves.append((curr_i, curr_j, next_move, min_cost))
  moves.append(next_move)

path = [from_idx]
for move in moves:
  curr_coords = hills_map[path[-1]][:-1]
  next_coords = curr_coords - move
  idx = next_coords[0]*N**2+next_coords[1]*N +next_coords[2]
  if idx in hex_idxs:
    path.append(idx)

In [55]:
path

[64620,
 62940,
 61260,
 59580,
 59540,
 57860,
 56180,
 56140,
 56100,
 54420,
 52740,
 52700,
 52660,
 52620,
 52580,
 52540,
 52500,
 52460,
 52420,
 52380,
 52340,
 52300,
 52260,
 52220,
 52180,
 52140]

In [56]:
visualize_solve(path)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1s_BSGk1yj7ZGX78AVfndzYyxSP0NBbdm)
</details>

### Обобщение задачки черепашки (1 балл)

Попробуем обобщить наш подход:
- Нужно научиться подбирать движения для двух точек
- Нужно научиться определять размерность задачи

Начнем с подбора движений

Для того, чтобы их подобрать, необходимо знать каким образом изменяются координаты м-ду стартовой и финишной точками

In [57]:
hills_map[from_idx] - hills_map[to_idx]

array([  7,  18, -25,   0], dtype=int32)

В задаче выше при движении от финала к старту
- x увеличивается
- y увеличивается
- z уменьшается

Для задачи выше будем брать те движения, где z уменьшается

В общем случае будем брать движения с фиксированной непопулярной динамикой для переменной

Например:
- $x\uparrow, y\uparrow, z\downarrow \Rightarrow$ Непопулярная динамика - вниз $\Rightarrow$ Берем движения где $z\downarrow$, "фиксируем" z
- $x\downarrow, y\uparrow, z\downarrow \Rightarrow$ Непопулярная динамика - вверх $\Rightarrow$ Берем движения где $y\uparrow$, "фиксируем" y

А как посчитать размерность?

Для каждого выбранного движения посчитаем разницу в изменяемых переменных

Посчитаем ручками на новом примере

In [58]:
new_from_idx = 42_140
new_to_idx = 64_300

In [59]:
def get_diff(idx_from, idx_to):
  return hills_map[idx_from][:3] - hills_map[idx_to][:3]

In [60]:
get_diff(new_from_idx, new_to_idx)

array([-13,  -8,  21], dtype=int32)

Берем движения где $z\uparrow$

In [61]:
movement_idxs = np.where(ALL_MOVEMENTS[:, 2] == 1)[0]

movements = ALL_MOVEMENTS[movement_idxs]

movements

array([[ 0, -1,  1],
       [-1,  0,  1]])

Выбросим $z$ из движений (т.к $z$ "зафиксирован")

In [62]:
movements[:, 0:2]

array([[ 0, -1],
       [-1,  0]])

Видим, что в первом движении изменяется $y$, а diff по $y$ = -8 $\Rightarrow$ Можно сделать 9 первых движений ((-8 / -1) + 1)

Аналогично, можно сделать 14 вторых движений

Значит размерность задачи (9, 14)

Теперь закодим эту историю

Начнем с функции получения возможных движений

In [63]:
"""
Функция возвращает движения для заданных индексов, а также индекс фиксированной переменной
"""
def get_movements(idx_from, idx_to):
  diff = get_diff(idx_from, idx_to)
  if diff[0] > 0 and diff[1] > 0 or diff[0] < 0 and diff[1] < 0:
    fixed_idx = 2
  elif diff[0] > 0 and diff[2] > 0 or diff[0] < 0 and diff[2] < 0:
    fixed_idx = 1
  elif diff[1] > 0 and diff[2] > 0 or diff[1] < 0 and diff[2] < 0:
    fixed_idx = 0

  if diff[fixed_idx] > 0:
    shift = 1
  else:
    shift = -1

  movement_idxs = np.where(ALL_MOVEMENTS[:, fixed_idx] == shift)[0]
  movements = ALL_MOVEMENTS[movement_idxs]
  return movements, fixed_idx

In [64]:
get_movements(new_from_idx, new_to_idx)

(array([[ 0, -1,  1],
        [-1,  0,  1]]),
 2)

Теперь сделаем функцию получения размерности задачи

In [65]:
"""
movements - массив движений, получаем из функции выше
fixed_var_idx - индекс зафиксированной переменной, получаем из функции выше
diff - разница координат, получаем из get_diff
"""
def get_size(movements, fixed_var_idx, diff):
    moves = diff / movements + 1
    sizes = []
    for i in range(moves.shape[0]):
      for j in range(moves.shape[-1]):
        if j == fixed_var_idx:
          continue
        if np.isinf(np.abs(moves[i, j])):
          continue
        sizes.append(int(moves[i, j]))
    return sizes

In [66]:
diff = get_diff(new_from_idx, new_to_idx)

movements, fixed_var_idx = get_movements(new_from_idx, new_to_idx)

get_size(movements, fixed_var_idx, diff)

<ipython-input-65-5cf856065df1>:7: RuntimeWarning:

divide by zero encountered in divide



[9, 14]

Теперь соберем функцию решения задачи задачи черепашки в общем случае

In [67]:
def solve_turtle_task(idx_from, idx_to):
  diff = get_diff(idx_from, idx_to)

  movements, fixed_var_idx = get_movements(idx_from, idx_to)

  size = get_size(movements, fixed_var_idx, diff)

  print(idx_from, idx_to)
  print(diff)
  print(movements)
  print(fixed_var_idx)
  print(size)

  part_n, part_m = size

  weights = np.full((part_n, part_m), PLAIN_TYPE)

  for i in range(part_n):
    for j in range(part_m):
        coords = hills_map[idx_from][:-1] - i * movements[0] + j - movements[1]
        idx = coords[0]*N**2+coords[1]*N + coords[2]
        weights[i, j] = hills_map[idx][-1]

  for i in range(part_n):
    for j in range(part_m):
        if i == 0 and j == 0:
          continue
        if i == 0:
          weights[i, j] +=  weights[i, j - 1]
        elif j == 0:
          weights[i, j] +=  weights[i - 1, j]
        else:
          weights[i, j] += min(weights[i - 1, j], weights[i, j - 1])
  curr_i = 0
  curr_j = 0

  n, m = weights.shape
  moves = []
  while curr_i != n - 1 or curr_j != m - 1:
    min_cost = 1e9
    next_move = None
    for move in movements:
      i, j = curr_i + abs(move[0]), curr_j + abs(move[1])
      if 0 <= i < n and 0<= j < m:
        cost = weights[i, j]
        if min_cost > cost:
          min_cost = cost
          next_move = move
    curr_i, curr_j = curr_i + abs(next_move[0]), curr_j + abs(next_move[1])
    moves.append((curr_i, curr_j))

  path = [idx_from]
  for i, j in moves:
    curr_coords = hills_map[idx_from][:-1]
    next_coords = curr_coords - i* movements[0] - j*movements[1]
    idx = next_coords[0]*N**2+next_coords[1]*N +next_coords[2]
    path.append(idx)
  return path

In [68]:
new_path = solve_turtle_task(new_from_idx, new_to_idx)

new_path

42140 64300
[-13  -8  21]
[[ 0 -1  1]
 [-1  0  1]]
2
[9, 14]


<ipython-input-65-5cf856065df1>:7: RuntimeWarning:

divide by zero encountered in divide



[42140,
 43820,
 45500,
 47180,
 48860,
 50540,
 52220,
 53900,
 55580,
 57260,
 58940,
 60620,
 62300,
 63980,
 64020,
 64060,
 64100,
 64140,
 64180,
 64220,
 64260,
 64300]

In [69]:
visualize_solve(new_path)

<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1kvcRtqG2lrODsnh06HQQFxoODxuNYJl5)
</details>

Сделаем функцию решения задачи черепашки и отрисовки пути

In [70]:
def draw_map_with_turtle_route(from_idx, to_idx):
  solve_path = solve_turtle_task(from_idx, to_idx)
  visualize_solve(solve_path)

In [71]:
draw_map_with_turtle_route(57_780, 52_420)

<ipython-input-65-5cf856065df1>:7: RuntimeWarning:

divide by zero encountered in divide



57780 52420
[  3   8 -11]
[[ 0  1 -1]
 [ 1  0 -1]]
2
[9, 4]


<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1ARqIWW1oFhdbXNHbusKbCz06WYSuX5dw)
</details>

In [72]:
draw_map_with_turtle_route(37_620, 51_500)

37620 51500
[ -8 -11  19]
[[ 0 -1  1]
 [-1  0  1]]
2
[12, 9]


<ipython-input-65-5cf856065df1>:7: RuntimeWarning:

divide by zero encountered in divide



<details>
  <summary>
    Если ячейка не загрузилась, должно быть так
  </summary>

  ![График](https://drive.google.com/uc?export=view&id=1eFkL6HfL42YLEfu-NhE1A8a5W1m2I8CH)
</details>

Динамическое программирование решает задачу достаточно хорошо, но ограничивает нас двумя возможными движениями за шаг алгоритма